In [1]:
import sys
# This is a workaround for running the script from the parent directory
sys.path.append('../')

# For loading sub file
import pysubs2

# For printing
import pprint

from langchain_openai import ChatOpenAI

In [2]:
import json
json_file = open("../config.json")
json_data = json.load(json_file)
api_key = json_data['openai_api_key']

## Scene Structure

In [3]:
ass_pass = "../sample/sample4.ass"
subs = pysubs2.load(ass_pass)
lines = "\n".join(str(line.text) for line in subs)
lines

'（誘導者の残響）招かれもせずに\\Nワシの玉座に入ったな\n（スロアン）ありえない…\n（誘導者の残響）ワシは深淵から蘇りし邪神\n最高誘導者だ\n（スロアン）残響だ！\nオリックスの記憶と繋がっているのか？\n（シヴ・アラス）兄よ！\n王よ！\n我が誘導者よ！\n戻ってこられたのか！\n（誘導者の残響）シヴ・アラスか\n我々の戦争はどうなったのだ？\n（シヴ・アラス）あなたは空の戦士に殺された\nそして、あなたの目前にいるのが\\Nその張本人だ！\n（誘導者の残響）このワシが殺されただと？\n（サバスン）残念ながら、噓ではない\nお前は死んだ神の記憶に過ぎない\nついになりたがっていた剣になることが\\N出来たようだな\n（誘導者の残響）ワシはアッカの死をもって即位した\n虫の亡骸から\\N玉座を造ったばかりだった\n（シヴ・アラス）あなたの殺害者を殺せば\nあなたの肉体は蘇る\nあなたの玉座は復活するのだ\n（誘導者の残響）破壊されたのであれば\nワシは滅するべきなのだ\nワシの継承者であり、\\Nワシを殺した者よ\n進み出るのだ\n（サバスン）二人とも大人げないぞ\n私たちはついに\\N自らの道を切り開くことができる\n（シヴ・アラス）ダメだ\n我々を再び見捨てることは許さない！\n（スロアン）こんなことは…ありえない！\n一度合流しよう\n早くそこから出るんだ！\n（シヴ・アラス）オリックス、私があなたの仇を討つ\n（誘導者の残響）刃を研ぐのだ\nその噓が\\Nワシの論理の真実となるように！\n（スロアン）行け\n（シヴ・アラス）なぜ兄は再び肉と骨の姿となり\n剣を手に持つことを拒むのだ？\n（誘導者の残響）ワシは深淵と交信した\nもしワシが本当に死んだのなら\nそれまでのことだ\nアイアット\nお前は欺瞞者に導かれるままに\n異端に堕ち\n鋭さを失った\nそんなお前が\\Nどんな真実を語ることができる？\n（シヴ・アラス）不相応な話し手は罰せられ\n偽りの偶像神は失墜し\n我は玉座を失った\nそしてあなたは死んだ\n我らの背後にあるものは\\N全て穢れている\n真実はこの先にある\n捧げもの\n剣\nそして我が王、オリックスが\nあなたはここに、\\N我と共にいる\nこれで我々は正しい道筋に\\N戻ることができる\n共に制服しようぞ\n（誘導者の残響）な

In [4]:
from src.logic.context import determine_scene_structure

llm = ChatOpenAI(model="gpt-4o", temperature=json_data['temperature'], api_key=api_key)

In [5]:
format_results = determine_scene_structure(llm, 'ja', 'en', lines)
pprint.pp(format_results)

('**Scene Type**: Interactive Dialogue  \n'
 '**Speaker Count**: 3+  \n'
 '**Interaction Style**: Characters engage in a dynamic exchange, responding '
 "to each other's lines and addressing each other directly, indicating a "
 'conflict and negotiation.  \n'
 '**Delivery Format**: Speaker-separated entries  \n'
 '**Notes**: The dialogue involves multiple characters, including "誘導者の残響," '
 '"スロアン," "シヴ・アラス," "サバスン," and "エリス・モーン," who interact with each other, '
 'revealing a complex narrative of betrayal, power struggles, and existential '
 'reflections.')


## Web Search

In [6]:
from langchain_community.tools.tavily_search.tool import TavilySearchResults
from src.logic.context import gather_context_from_web

In [7]:
import os
os.environ["TAVILY_API_KEY"] = json_data["tavily_api_key"]
search_tool = TavilySearchResults(k=3)

In [8]:
web_search_results = gather_context_from_web(llm, search_tool, output_lang="en",
                                             series_name="destiny 2", keywords="episode heresy", transcript=lines)

pprint.pp(web_search_results)

('**Character Relationships:** The episode "Heresy" in Destiny 2 involves key '
 'characters such as Eris Morn, the Drifter, and Deputy Commander Sloane. Eris '
 'Morn has a significant role, working with the Guardian to oppose the Hive '
 'and imprison the Echo of Navigation and the memory of Oryx in her Throne '
 'World. There is also a notable reunion between Eris and the Drifter.\n'
 '\n'
 '**Story Premise and Genre:** "Heresy" is part of the Destiny 2 series and is '
 'set in a science fiction universe. The narrative focuses on a confrontation '
 'with the Hive pantheon, including characters like Oryx, Savathûn, and Xivu '
 'Arath, and involves returning to the Dreadnaught ship to combat Dire Taken '
 'forces. The story is rich in lore, exploring themes of power and opposition '
 'against dark forces.')


## Character Identification

In [18]:
from src.logic.context import identify_characters

In [19]:
characters_output = identify_characters(llm, input_lang="ja", output_lang="en",
                                        transcript=lines, format_description=format_results, web_context=web_search_results)

pprint.pp(characters_output)

('- **誘導者の残響**: Appears to be the spirit or echo of a once powerful entity, '
 'possibly a god, who is now questioning their existence and role. Engages in '
 'philosophical debate about power and existence. [Narrative Focus]\n'
 '- **スロアン**: Expresses disbelief and urgency, possibly a character reacting '
 'to the unfolding events and urging others to action.\n'
 '- **シヴ・アラス**: Appears to be a loyal follower or sibling of the "誘導者の残響," '
 'advocating for revenge and restoration of power. Engages in intense dialogue '
 'about loyalty and betrayal.\n'
 '- **サバスン**: A character who challenges and confronts the beliefs of others, '
 'possibly with a manipulative or insightful demeanor. Engages in debates '
 'about truth and freedom.\n'
 '- **エリス・モーン**: Provides commentary on the situation, possibly an observer or '
 "someone with insight into the characters' motivations and flaws.")


## Summarize

In [20]:
from src.logic.context import summarize_scene

In [21]:
summary_output = summarize_scene(llm, input_lang="ja", output_lang="en",
                                 transcript=lines, format_description=format_results, 
                                 web_context=web_search_results, character_list=characters_output)

pprint.pp(summary_output)

('**Setting**: The scene takes place in a throne room or a similar regal and '
 'significant location, possibly within a realm associated with the Hive or '
 'the Abyss. The environment suggests a place of power and authority, where '
 'significant confrontations and revelations occur.\n'
 '\n'
 '**Main Characters**:  \n'
 '- **誘導者の残響 (Echo of the Guide)**: The spirit or essence of a once-powerful '
 'god, likely Oryx, who is grappling with their identity and existence.\n'
 '- **スロアン (Sloane)**: A character expressing disbelief and urgency, likely '
 'reacting to the unfolding events and urging others to leave or regroup.\n'
 '- **シヴ・アラス (Siv Arath)**: A loyal follower or sibling of the Echo, focused '
 'on revenge and restoring power, engaging in intense dialogue about loyalty '
 'and betrayal.\n'
 '- **サバスン (Savathûn)**: A manipulative or insightful figure challenging '
 "others' beliefs, engaging in debates about truth and freedom.\n"
 '- **エリス・モーン (Eris Morn)**: An observer or comm

## Tone

In [13]:
from src.logic.context import determine_tone

In [14]:
tone_output = determine_tone(llm, input_lang='ja', output_lang='en',
                             transcript=lines, format_description=format_results,
                             web_context=web_search_results)

pprint.pp(tone_output)

('The overall tone of the scene is casual and comedic, with a playful and '
 'teasing dynamic between the characters. The speech is informal, '
 'characterized by blunt exchanges, light-hearted banter, and occasional '
 'sarcasm, which suggests a close and familiar relationship between the '
 'speakers.')
